In [1]:
import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.datasets.coco import CocoDetection
from torchvision import transforms
from torch.utils.data import DataLoader
import utils
from engine import train_one_epoch, evaluate
from torch.utils.tensorboard import SummaryWriter

In [2]:
model = fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=True)

In [ ]:
dataset = CocoDetection('val2017', 'annotations/instances_val2017.json',transform=transforms.ToTensor())
print(dataset[0][0].shape)

In [ ]:
print(dataset[0][1])

In [ ]:
print(dataset[0][1][0]['bbox'])

In [3]:
class DetectDataset(CocoDetection):
    def __init__(self, root, annFile) -> None:
        super().__init__(root, annFile, transforms.ToTensor())
        self.novel_cls = [1, 17, 16, 21, 18, 19, 20, 2, 9, 6, 3, 4, 7, 44, 62, 67, 64, 63, 72]
        self.base_cls = [i for i in range(1,91) if i not in self.novel_cls]
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        if target is None or len(target)==0:
            return None, None
        nTarget = {'boxes':[], 'labels':[], 'image_id':target[0]['image_id']}
        for t in target:
            box = t['bbox'].copy()
            box[2] += box[0]
            box[3] += box[1]
            if box[2]-box[0] < 1 or box[3]-box[1] < 1:
                continue
            cate = t['category_id']
            if cate in self.novel_cls:
                continue
            nTarget['boxes'].append(box)
            nTarget['labels'].append(cate)
        nTarget = {key:torch.tensor(val) for key, val in nTarget.items()}
        return img, nTarget
    def __len__(self):
        return super().__len__()

In [4]:
valset = DetectDataset('val2017', 'annotations/instances_val2017.json')
trainset = DetectDataset('train2017', 'annotations/instances_train2017.json')
valset[5]

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!
loading annotations into memory...
Done (t=15.64s)
creating index...
index created!


(tensor([[[0.7961, 0.7961, 0.7961,  ..., 0.8588, 0.8588, 0.8588],
          [0.7961, 0.8000, 0.8000,  ..., 0.8549, 0.8549, 0.8549],
          [0.8039, 0.8039, 0.8039,  ..., 0.8588, 0.8588, 0.8588],
          ...,
          [0.8902, 0.8667, 0.8314,  ..., 0.7647, 0.7725, 0.7765],
          [0.8627, 0.8588, 0.8588,  ..., 0.7451, 0.7451, 0.7490],
          [0.8510, 0.8353, 0.8588,  ..., 0.7294, 0.7373, 0.7412]],
 
         [[0.8078, 0.8078, 0.8078,  ..., 0.8588, 0.8588, 0.8588],
          [0.8078, 0.8118, 0.8118,  ..., 0.8549, 0.8549, 0.8549],
          [0.8157, 0.8157, 0.8157,  ..., 0.8588, 0.8588, 0.8588],
          ...,
          [0.8902, 0.8667, 0.8314,  ..., 0.7765, 0.7843, 0.7882],
          [0.8627, 0.8588, 0.8549,  ..., 0.7569, 0.7569, 0.7608],
          [0.8471, 0.8314, 0.8627,  ..., 0.7451, 0.7529, 0.7608]],
 
         [[0.8353, 0.8353, 0.8353,  ..., 0.8667, 0.8667, 0.8667],
          [0.8353, 0.8392, 0.8392,  ..., 0.8627, 0.8627, 0.8627],
          [0.8431, 0.8431, 0.8431,  ...,

In [5]:
val = DataLoader(valset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)
train = DataLoader(trainset,batch_size = 1, shuffle=True, collate_fn=utils.collate_fn)

In [ ]:
print(next(iter(val))[1])

In [ ]:
img, labs = next(iter(val))
print(img[0].shape)
print(labs[0])

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.load_state_dict(torch.load('fasterrcnn1.weights'))
writer = SummaryWriter()
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.0001)
#optimizer = torch.optim.SGD(params, lr=0.0002, momentum=0.9, weight_decay=0.0001)
#model.load_state_dict(torch.load('fasterrcnn0.weights'))
num_epochs = 18
for epoch in range(num_epochs):
    train_one_epoch(writer, model, optimizer, val, device, epoch, print_freq=1000)
    evaluate(writer, model, val, device, print_freq=1000)
    torch.save(model.state_dict(), f'fasterrcnn1.weights')

Error: Kernel is dead

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params,lr=0.0001)
#optimizer = torch.optim.SGD(params, lr=0.02, momentum=0.9, weight_decay=0.0001)
#model.load_state_dict(torch.load('fasterrcnn0.weights'))
num_epochs = 18
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train, device, epoch, print_freq=10000)
    evaluate(model, val, device, print_freq=1000)
    torch.save(model.state_dict(), f'fasterrcnn2.weights')

Error: Kernel is dead

In [ ]:
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
evaluate(model, val, device)

In [ ]:
img, lab = next(iter(val))
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.to(device)
model.eval()
pred = model([im.to(device) for im in img])
print(pred)
print(lab)